# 1. Imports

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf

# 2. Definitions

**2.1 - Data**

2.1.1 - Pipeline

In [32]:
#INPUT

#Location level
TRAIN_DATA_DIR = 'Data/training/'
VALIDATION_DATA_DIR = 'Data/validation'

#Sample level
TRAIN_SAMPLES = 1098
VALIDATION_SAMPLES = 272

#Image level
IMG_WIDTH, IMG_HEIGHT = 224, 224


#PROCESSING
BATCH_SIZE = 16

#OUTPUT
NUM_CLASSES = 10


2.1.2 - Loading and Augmentation

In [33]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [34]:
from tensorflow.keras.applications.mobilenet import preprocess_input

In [35]:
train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,
                               rotation_range=45,
                               width_shift_range=0.3,
                               height_shift_range=0.3,
                               horizontal_flip=True,
                               fill_mode="nearest")

In [36]:
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [37]:
train_generator = train_datagen.flow_from_directory(TRAIN_DATA_DIR,
                                                   target_size=(IMG_WIDTH,IMG_HEIGHT),
                                                   batch_size = BATCH_SIZE,
                                                   class_mode = 'categorical')

Found 1098 images belonging to 10 classes.


In [38]:
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DATA_DIR,
                                                      target_size = (IMG_WIDTH,IMG_HEIGHT),
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = False,
                                                      class_mode = 'categorical')

Found 272 images belonging to 10 classes.


**2 - Model**

2.1 - Define 

In [39]:
from tensorflow.keras.applications.mobilenet import MobileNet

In [40]:
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D

In [41]:
from tensorflow.keras.models import Model

INPUT

In [42]:
#Instantiate Model
mobilenet = MobileNet(include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

In [43]:
# Freeze Layers
for layer in mobilenet.layers:
        layer.trainable=False

In [44]:
# Check the layers

for (i,layer) in enumerate(mobilenet.layers):
    print(str(i)+" "+layer.__class__.__name__,layer.trainable)

0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 ReLU False
11 ZeroPadding2D False
12 DepthwiseConv2D False
13 BatchNormalization False
14 ReLU False
15 Conv2D False
16 BatchNormalization False
17 ReLU False
18 DepthwiseConv2D False
19 BatchNormalization False
20 ReLU False
21 Conv2D False
22 BatchNormalization False
23 ReLU False
24 ZeroPadding2D False
25 DepthwiseConv2D False
26 BatchNormalization False
27 ReLU False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 DepthwiseConv2D False
32 BatchNormalization False
33 ReLU False
34 Conv2D False
35 BatchNormalization False
36 ReLU False
37 ZeroPadding2D False
38 DepthwiseConv2D False
39 BatchNormalization False
40 ReLU False
41 Conv2D False
42 BatchNormalization False
43 ReLU False
44 DepthwiseConv2D False
45 BatchNormalization False
46 ReLU False
47 Conv2D False
48 Batc

In [45]:
# Define the layers

def model_maker(base_model, num_classes):
        custom_model = base_model.output
        custom_model = GlobalAveragePooling2D()(custom_model)
        custom_model = Dense(1024, activation = 'relu')(custom_model)
        custom_model = Dense(1024, activation = 'relu')(custom_model)
        custom_model = Dense(512, activation = 'relu')(custom_model)
        predictions = Dense(NUM_CLASSES, activation = 'softmax')(custom_model)
        return predictions
        
        
        
        

# Training

In [46]:
model = model_maker(mobilenet,NUM_CLASSES)
model = Model(inputs=mobilenet.input, outputs=model)
print(model.summary())


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [47]:
%time model.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(0.001),metrics = ['acc'])

# Train the model

%time model.fit_generator(train_generator, steps_per_epoch = math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE),epochs = 10,validation_data = validation_generator, validation_steps = math.ceil(float(VALIDATION_SAMPLES) / BATCH_SIZE))

model.save('model.h5')

CPU times: user 47.6 ms, sys: 3.37 ms, total: 51 ms
Wall time: 49.6 ms
Epoch 1/10
69/69 [==============================] - 251s 4s/step - loss: 1.3780 - acc: 0.5437 - val_loss: 0.4878 - val_acc: 0.8382
Epoch 2/10
69/69 [==============================] - 242s 4s/step - loss: 0.6584 - acc: 0.7869 - val_loss: 0.3559 - val_acc: 0.8897
Epoch 3/10
69/69 [==============================] - 241s 3s/step - loss: 0.4243 - acc: 0.8452 - val_loss: 0.3447 - val_acc: 0.8750
Epoch 4/10
69/69 [==============================] - 241s 3s/step - loss: 0.3687 - acc: 0.8698 - val_loss: 0.2505 - val_acc: 0.9301
Epoch 5/10
69/69 [==============================] - 240s 3s/step - loss: 0.3172 - acc: 0.8934 - val_loss: 0.2125 - val_acc: 0.9412
Epoch 6/10
69/69 [==============================] - 243s 4s/step - loss: 0.3371 - acc: 0.8825 - val_loss: 0.2140 - val_acc: 0.9412
Epoch 7/10
69/69 [==============================] - 245s 4s/step - loss: 0.3613 - acc: 0.8834 - val_loss: 0.1320 - val_acc: 0.9632
Epoch 8/10
6